

**Softmax convex conjugate:**
$$
f^*_{\varepsilon}(y) := \varepsilon \log \left(\int_{\text{dom} f} \exp \left(\frac{1}{\varepsilon} \{\langle y,x \rangle - f(x)\}\right) dx \right),
$$
which converges to $f^*$ as $\varepsilon \downarrow 0$. Peyr\'e (2020) has shown that $f^*_{\varepsilon}$ can be expressed via Gaussian smoothing as
$$
f^*_{\varepsilon}(y) = Q^{-1}_{\varepsilon} \left(\frac{1}{Q_{\varepsilon}(f) * G_{\varepsilon}}\right)(y),
$$
where
$$
G_{\varepsilon}(x) = \exp \left(- \frac{1}{2 \varepsilon} \|x\|_2^2\right) \quad \text{and} \quad
Q_{\varepsilon}(f) = \exp \left(\frac{1}{2 \varepsilon} \|x\|_2^2 - \frac{1}{\varepsilon} f(x)\right).
$$

$$
Q^{-1}_{\varepsilon}\{F\} := \frac{1}{2} \|\cdot\|^2 - \varepsilon \log(F(\cdot)).
$$

MCMC implementation:

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
import matplotlib.colors as colors
import psutil
import os
import gc
import pandas as pd
from tabulate import tabulate
import tracemalloc
from scipy.stats import qmc

# ---------- Memory tracking functions ------------------------------
def start_memory_tracking():
    """Start tracking memory allocations"""
    gc.collect()  # Force garbage collection
    tracemalloc.start()
    return tracemalloc.get_traced_memory()[0] / (1024 * 1024)  # Current memory in MB

def get_peak_memory():
    """Get peak memory usage in MB since tracking started"""
    current, peak = tracemalloc.get_traced_memory()
    return peak / (1024 * 1024)  # Peak memory in MB

def stop_memory_tracking():
    """Stop tracking memory allocations"""
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    return peak / (1024 * 1024)  # Peak memory in MB

def free_memory():
    """Aggressively free memory"""
    gc.collect()  # Collect garbage
    if hasattr(plt, 'close'):
        plt.close('all')  # Close all matplotlib figures

    # Try to clear large variables from memory
    for name in list(globals().keys()):
        if name.startswith('__'):
            continue
        obj = globals()[name]
        if isinstance(obj, np.ndarray) and obj.size > 1000000:
            del globals()[name]

    gc.collect()  # Collect garbage again after deleting variables

# ---------- Test functions -----------------------------------------
def neg_log(d):
    """d-dimensional negative logarithm function: u(x) = -sum(log(x_i))
    Domain: x_i > 0
    LF transform: u*(s) = -d - sum(log(-s_i)) for s_i < 0
    """
    def func(*args):
        # Handle domain constraints - inputs should be positive
        result = 0
        for x in args:
            x_safe = np.maximum(x, 1e-10)  # Avoid log(0)
            result -= np.log(x_safe)
        return result
    return func

def neg_entropy(d):
    """d-dimensional negative entropy function: u(x) = sum(x_i * log(x_i))
    Domain: x_i > 0
    LF transform: u*(s) = sum(exp(s_i - 1))
    """
    def func(*args):
        result = 0
        for x in args:
            # Handle domain constraints and avoid singularities
            x_safe = np.maximum(x, 1e-10)
            x_log_x = x_safe * np.log(x_safe)
            # Set values to 0 for x close to 0 (lim x→0 x log(x) = 0)
            x_log_x = np.where(x_safe < 1e-8, 0, x_log_x)
            result += x_log_x
        return result
    return func

# ---------- Analytical LF transforms for validation ----------------
def neg_log_transform_analytical(s_arrs):
    """
    Analytical LF transform for negative log function:
    f*(s) = -d - sum(log(-s_i)) for s_i < 0
    """
    d = len(s_arrs)
    S = np.meshgrid(*s_arrs, indexing='ij', sparse=False)

    # Initialize with -d term
    result = np.full(tuple(len(s) for s in s_arrs), -d, dtype=float)

    # Add log terms for each dimension
    for i in range(d):
        # Apply only where s < 0 (domain constraint)
        valid_mask = S[i] < 0
        # Set large negative values for invalid regions
        log_term = np.log(-S[i])
        result = np.where(valid_mask, result - log_term, -np.inf)

    return result

def neg_entropy_transform_analytical(s_arrs):
    """
    Analytical LF transform for negative entropy function:
    f*(s) = sum(exp(s_i - 1))
    """
    d = len(s_arrs)
    S = np.meshgrid(*s_arrs, indexing='ij', sparse=False)

    # Initialize with zeros
    result = np.zeros(tuple(len(s) for s in s_arrs), dtype=float)

    # Add exp terms for each dimension
    for i in range(d):
        result += np.exp(S[i] - 1)

    return result

# ---------- Monte Carlo LF Transform Methods ----------------------
def lf_transform_monte_carlo(x_ranges, f, s_arrs, eps=0.1, n_samples=100000, method='quasi'):
    """
    Monte Carlo approximation of Legendre-Fenchel transform.

    Parameters:
    x_ranges : list of tuples
        Range for each dimension of x (min, max)
    f : function
        The function to transform
    s_arrs : list of arrays
        Grid points in dual space (one array per dimension)
    eps : float
        Smoothing parameter
    n_samples : int
        Number of Monte Carlo samples
    method : str
        Monte Carlo method to use: 'basic', 'quasi', 'importance', or 'adaptive'

    Returns:
    numpy.ndarray
        The approximate Legendre-Fenchel transform f*_eps(s)
    """
    # Get dimension
    d = len(x_ranges)

    # Create output array for the transform
    result = np.empty([len(s) for s in s_arrs])

    # Create separate error estimates array if needed
    error_estimates = None
    if method in ['basic', 'quasi']:
        error_estimates = np.empty_like(result)

    # Calculate domain volume
    domain_volume = np.prod([x_max - x_min for x_min, x_max in x_ranges])

    # Generate samples based on the selected method
    if method == 'basic':
        # Simple uniform random sampling
        samples = np.array([np.random.uniform(x_min, x_max, n_samples)
                           for x_min, x_max in x_ranges]).T

    elif method == 'quasi':
        # Quasi-Monte Carlo with Sobol sequences for better coverage
        # Initialize Sobol sequence generator
        sampler = qmc.Sobol(d=d, scramble=True)

        # Generate samples in [0, 1]
        unit_samples = sampler.random(n_samples)

        # Scale to the domain range
        samples = np.zeros((n_samples, d))
        for i in range(d):
            x_min, x_max = x_ranges[i]
            samples[:, i] = unit_samples[:, i] * (x_max - x_min) + x_min

    elif method == 'importance' or method == 'adaptive':
        # For importance sampling, we need an initial set of samples to identify high-contribution regions
        # Start with uniform samples
        initial_samples = np.array([np.random.uniform(x_min, x_max, min(10000, n_samples // 10))
                                   for x_min, x_max in x_ranges]).T

        # Pre-compute function values
        f_values_initial = np.array([f(*x) for x in initial_samples])

        # For each output point, we'll generate specialized importance samples
        # This makes this method more computationally intensive but potentially more accurate
        samples_per_point = max(1000, n_samples // (np.prod([len(s) for s in s_arrs])))

    else:
        raise ValueError(f"Unknown Monte Carlo method: {method}")

    # For basic and quasi-Monte Carlo, pre-compute function values for all samples
    if method in ['basic', 'quasi']:
        f_values = np.array([f(*x) for x in samples])

    # Iterate through the output grid
    it = np.nditer(result, flags=['multi_index'], op_flags=['writeonly'])
    while not it.finished:
        # Get current slope point
        s_point = [s_arrs[i][it.multi_index[i]] for i in range(d)]

        if method in ['basic', 'quasi']:
            # Compute inner products for all samples
            inner_products = np.sum([s_point[i] * samples[:, i] for i in range(d)], axis=0)

            # Compute exponents
            exponents = (inner_products - f_values) / eps

            # Shift to prevent overflow
            max_exp = np.max(exponents)
            exponents -= max_exp

            # Compute Monte Carlo approximation
            mc_sum = np.sum(np.exp(exponents))
            mc_avg = mc_sum / n_samples

            # Apply scaling and logarithm
            it[0] = eps * (np.log(domain_volume * mc_avg) + max_exp)

            # Calculate error estimate (for standard Monte Carlo)
            # Standard error = sigma / sqrt(N)
            if error_estimates is not None and method == 'basic':
                variance = np.var(np.exp(exponents)) * (domain_volume**2)
                std_error = np.sqrt(variance / n_samples)
                error_estimates[it.multi_index] = eps * std_error / (domain_volume * mc_avg)


        elif method == 'importance':
            # For importance sampling, we want to sample more heavily in regions where the integrand is large
            # Compute integrand values for initial samples for the current s_point
            inner_products_init = np.sum([s_point[i] * initial_samples[:, i] for i in range(d)], axis=0)
            integrand_values = np.exp((inner_products_init - f_values_initial) / eps)

            # Normalize to create a probability distribution
            if np.sum(integrand_values) > 0:
                prob_dist = integrand_values / np.sum(integrand_values)
            else:
                # If all values are zero (underflow), use uniform distribution
                prob_dist = np.ones_like(integrand_values) / len(integrand_values)

            # Generate importance samples for this specific s_point
            importance_indices = np.random.choice(
                range(len(initial_samples)),
                size=samples_per_point,
                replace=True,
                p=prob_dist
            )
            importance_samples = initial_samples[importance_indices]

            # Compute function values for importance samples
            f_values_importance = f_values_initial[importance_indices]

            # Compute integrand with importance sampling correction
            inner_products = np.sum([s_point[i] * importance_samples[:, i] for i in range(d)], axis=0)
            exponents = (inner_products - f_values_importance) / eps

            # Apply importance sampling correction: divide by probability used for sampling
            importance_weights = 1.0 / (prob_dist[importance_indices] * len(prob_dist))

            # Avoid overflow
            max_exp = np.max(exponents)
            exponents -= max_exp
            weighted_sum = np.sum(np.exp(exponents) * importance_weights)

            # Compute final result
            it[0] = eps * (np.log(domain_volume * weighted_sum / samples_per_point) + max_exp)

        elif method == 'adaptive':
            # Similar to importance sampling but with adaptive refinement
            # First pass with initial samples
            inner_products_init = np.sum([s_point[i] * initial_samples[:, i] for i in range(d)], axis=0)
            integrand_values = np.exp((inner_products_init - f_values_initial) / eps)

            # Identify regions needing refinement (high integrand value or high variance)
            if len(integrand_values) > 0:
                threshold = np.percentile(integrand_values, 95)  # Focus on top 5%
                high_contribution_indices = np.where(integrand_values > threshold)[0]

                if len(high_contribution_indices) > 0:
                    # Extract high-contribution samples
                    high_samples = initial_samples[high_contribution_indices]

                    # Define a refined sampling region around these samples
                    refined_ranges = []
                    for dim in range(d):
                        min_val = max(x_ranges[dim][0], np.min(high_samples[:, dim]) - 0.1 * (x_ranges[dim][1] - x_ranges[dim][0]))
                        max_val = min(x_ranges[dim][1], np.max(high_samples[:, dim]) + 0.1 * (x_ranges[dim][1] - x_ranges[dim][0]))
                        refined_ranges.append((min_val, max_val))

                    # Generate refined samples
                    refined_samples = np.array([np.random.uniform(r_min, r_max, samples_per_point)
                                              for r_min, r_max in refined_ranges]).T

                    # Compute function values for refined samples
                    f_values_refined = np.array([f(*x) for x in refined_samples])

                    # Compute integrand for refined samples
                    inner_products_refined = np.sum([s_point[i] * refined_samples[:, i] for i in range(d)], axis=0)
                    exponents_refined = (inner_products_refined - f_values_refined) / eps

                    # Calculate refined volume
                    refined_volume = np.prod([r_max - r_min for r_min, r_max in refined_ranges])
                    ratio = refined_volume / domain_volume

                    # Combine with initial estimate (weighted by volume ratio)
                    max_exp_init = np.max(inner_products_init - f_values_initial) / eps if len(inner_products_init) > 0 else 0
                    init_sum = np.sum(np.exp(((inner_products_init - f_values_initial) / eps) - max_exp_init))

                    max_exp_refined = np.max(exponents_refined) if len(exponents_refined) > 0 else 0
                    refined_sum = np.sum(np.exp(exponents_refined - max_exp_refined))

                    # Combine estimates with appropriate volume scaling
                    max_exp_combined = max(max_exp_init, max_exp_refined)
                    init_contribution = init_sum * np.exp(max_exp_init - max_exp_combined) * (1 - ratio)
                    refined_contribution = refined_sum * np.exp(max_exp_refined - max_exp_combined) * ratio

                    combined_sum = init_contribution + refined_contribution
                    it[0] = eps * (np.log(domain_volume * combined_sum / samples_per_point) + max_exp_combined)
                else:
                    # Fall back to basic MC if no high-contribution regions found
                    max_exp = np.max((inner_products_init - f_values_initial) / eps)
                    mc_sum = np.sum(np.exp(((inner_products_init - f_values_initial) / eps) - max_exp))
                    mc_avg = mc_sum / len(initial_samples)
                    it[0] = eps * (np.log(domain_volume * mc_avg) + max_exp)
            else:
                # No valid samples, fall back to analytical approximation if possible
                it[0] = 0  # Default value

        it.iternext()

    # Return both result and error estimates if available
    if error_estimates is not None:
        return result, error_estimates
    return result

# ---------- Error calculation functions ---------------------
def calculate_errors(U_mc, U_ana, validation_set=None):
    """
    Calculate various error metrics between Monte Carlo approximation and analytical solution.

    Parameters:
    U_mc : numpy.ndarray
        Monte Carlo approximation
    U_ana : numpy.ndarray
        Analytical solution
    validation_set : tuple, optional
        Tuple of slice objects defining a validation set region

    Returns:
    dict
        Dictionary of error metrics
    """
    # Get valid points (exclude -inf values)
    valid_mask = ~np.isinf(U_ana) & ~np.isinf(U_mc) & ~np.isnan(U_ana) & ~np.isnan(U_mc)

    if np.sum(valid_mask) == 0:
        return {
            'max': np.nan,
            'mean': np.nan,
            'median': np.nan,
            'max_validation': np.nan,
            'rmse': np.nan,
            'valid_percentage': 0
        }

    # Extract valid points
    U_mc_valid = U_mc[valid_mask]
    U_ana_valid = U_ana[valid_mask]

    # Calculate absolute errors
    abs_errors = np.abs(U_mc_valid - U_ana_valid)

    # Calculate max error over validation set if provided
    max_validation_error = np.nan
    if validation_set is not None:
        # Apply validation set slices
        validation_mask = valid_mask.copy()
        if isinstance(validation_set, tuple):
            validation_mask[validation_set] &= valid_mask[validation_set]

            # Check if there are valid points in the validation set
            if np.any(validation_mask):
                # Calculate max error over validation set
                validation_errors = np.abs(U_mc[validation_mask] - U_ana[validation_mask])
                max_validation_error = np.max(validation_errors)
    else:
        # If no validation set provided, use a central region
        # (exclude 20% of points from each edge)
        shape = valid_mask.shape
        ndim = len(shape)

        validation_mask = valid_mask.copy()
        for d in range(ndim):
            size = shape[d]
            margin = max(1, int(size * 0.2))

            # Create a slice for this dimension
            idx = tuple(slice(margin, size - margin) if i == d else slice(None)
                        for i in range(ndim))

            # Update validation mask
            subregion = np.zeros_like(valid_mask, dtype=bool)
            subregion[idx] = True
            validation_mask &= subregion

        # Calculate max error over validation set
        if np.any(validation_mask):
            validation_errors = np.abs(U_mc[validation_mask] - U_ana[validation_mask])
            max_validation_error = np.max(validation_errors)

    # Calculate RMSE
    squared_errors = (U_mc_valid - U_ana_valid)**2
    rmse = np.sqrt(np.mean(squared_errors))  # Root Mean Square Error

    # Calculate percentage of valid points
    valid_percentage = 100.0 * np.sum(valid_mask) / valid_mask.size

    # Free memory for large arrays
    del validation_mask, valid_mask
    gc.collect()

    return {
        'max': np.max(abs_errors),
        'mean': np.mean(abs_errors),
        'median': np.median(abs_errors),
        'max_validation': max_validation_error,
        'rmse': rmse,
        'valid_percentage': valid_percentage
    }

# ---------- Plot functions -----------------------------------------
def plot_1d_results(x_ranges, f, s_arrays, u_star_mc, u_star_analytical, eps, func_name, mc_method, n_samples):
    """Plot results for 1D transform."""
    plt.figure(figsize=(15, 5))

    # Sample points to plot original function
    x = np.linspace(x_ranges[0][0], x_ranges[0][1], 100)
    u_x = np.array([f(xi) for xi in x])

    # Plot original function
    plt.subplot(1, 3, 1)
    plt.plot(x, u_x)
    plt.xlabel('x')
    plt.ylabel('u(x)')
    plt.title(f'Original {func_name.capitalize()} Function')
    plt.grid(True)

    # Plot transforms
    plt.subplot(1, 3, 2)
    s = s_arrays[0]
    plt.plot(s, u_star_mc, 'g-.', label=f'{mc_method.capitalize()} MC (ε={eps}, N={n_samples})')
    plt.plot(s, u_star_analytical, 'k-', label='Analytical')

    plt.xlabel('s')
    plt.ylabel('u*(s)')
    plt.title('Legendre Transform Comparison')
    plt.legend()
    plt.grid(True)

    # Plot error
    plt.subplot(1, 3, 3)
    error = np.abs(u_star_mc - u_star_analytical)
    plt.semilogy(s, error, 'r-', label='|MC-Analytical|')

    # Add reference line showing MC error scaling O(1/√N)
    ref_line = 1.0 / np.sqrt(n_samples) * np.ones_like(s)
    plt.semilogy(s, ref_line, 'k--', label=f'1/√N reference (N={n_samples})')

    plt.xlabel('s')
    plt.ylabel('Absolute Error')
    plt.title('Error vs Analytical')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()

# ---------- Run comprehensive benchmark ---------------------------
def run_comprehensive_benchmark(func_name, dimensions=[1, 2, 3, 4, 5], eps_values=[0.001, 0.01, 0.1, 0.5],
                               verbose=True, columns_to_display=None, mc_method='quasi', n_samples=100000):
    """
    Run comprehensive benchmark for the specified function with multiple dimensions and epsilon values.

    Parameters:
    func_name : str
        Name of the function to test ('neg_log' or 'neg_entropy')
    dimensions : list of int
        List of dimensions to test
    eps_values : list of float
        List of epsilon values to test
    verbose : bool
        Whether to print progress information
    columns_to_display : list of str
        List of column names to display in the final table
    mc_method : str
        Monte Carlo method to use ('basic', 'quasi', 'importance', 'adaptive')
    n_samples : int
        Number of Monte Carlo samples to use

    Returns:
    pandas.DataFrame
        DataFrame containing benchmark results
    """
    # Select function based on name
    if func_name == 'neg_log':
        f_creator = neg_log
        analytical_transform = neg_log_transform_analytical
        # Domain constraints: x > 0, s < 0
        x_range = (0.1, 5.0)
        s_range = (-5.0, -0.1)
    elif func_name == 'neg_entropy':
        f_creator = neg_entropy
        analytical_transform = neg_entropy_transform_analytical
        # Domain constraints: x > 0, s can be any real number
        x_range = (0.1, 5.0)
        s_range = (-3.0, 3.0)
    else:
        raise ValueError(f"Unknown function type: {func_name}")

    # Create results list to convert to DataFrame later
    results_list = []

    # If no columns specified, use default set
    if columns_to_display is None:
        columns_to_display = ['d', 'eps', 'sample_size', 'time', 'memory',
                             'max_err', 'max_validation', 'mean_err', 'rmse', 'conv_rate']

    # Print header
    if verbose:
        print(f"\n===== BENCHMARK FOR {func_name.upper()} (MONTE CARLO: {mc_method.upper()}, {n_samples} SAMPLES) =====\n")
        header = f"{'d':>2} | {'eps':>6} | {'sample size':>12} | {'time(s)':>8} | {'MB':>6} | {'max err':>8} | {'max val':>8} | {'mean err':>8} | {'RMSE':>8} | {'conv rate':>9}"
        print(header)
        print("-" * len(header))

    # Previous errors for convergence rate calculation
    prev_eps = {}
    prev_errors = {}

    # Loop through dimensions
    for d in dimensions:
        # Create evaluation grids for output and analytical comparison
        n_eval_pts = min(20, 100 // d)  # Reduce grid size for high dimensions
        s_arrs = [np.linspace(s_range[0], s_range[1], n_eval_pts) for _ in range(d)]
        x_ranges = [(x_range[0], x_range[1]) for _ in range(d)]
        f = f_creator(d)

        # Calculate analytical solution for comparison
        try:
            # Calculate analytical solution
            U_ana = analytical_transform(s_arrs)

            # Add a horizontal rule before each dimension
            if d > dimensions[0] and verbose:
                print("-" * len(header))

            # Loop through epsilon values
            for eps in sorted(eps_values):
                try:
                    # Measure memory and time using tracemalloc
                    start_memory_tracking()

                    t0 = time.perf_counter()

                    # Use Monte Carlo method - handle both result and possible error estimates
                    result_data = lf_transform_monte_carlo(
                        x_ranges=x_ranges,
                        f=f,
                        s_arrs=s_arrs,
                        eps=eps,
                        n_samples=n_samples,
                        method=mc_method
                    )

                    # Handle either single result or (result, error_estimates) tuple
                    if isinstance(result_data, tuple) and len(result_data) == 2:
                        U_mc, error_estimates = result_data
                    else:
                        U_mc = result_data
                        error_estimates = None

                    t_mc = time.perf_counter() - t0

                    # Get peak memory usage
                    peak_memory = get_peak_memory()
                    stop_memory_tracking()

                    # Define validation set (central region)
                    validation_set = None  # Using default central region

                    # Calculate errors
                    errors = calculate_errors(U_mc, U_ana, validation_set)

                    # Calculate convergence rate if possible
                    conv_rate = np.nan
                    if d in prev_eps and d in prev_errors and prev_eps[d] is not None:
                        if prev_errors[d] > 0 and errors['max_validation'] > 0:  # Avoid division by zero or log(0)
                            conv_rate = np.log(errors['max_validation']/prev_errors[d]) / np.log(eps/prev_eps[d])

                    # Store results in dictionary
                    size_info = f"MC({n_samples})"

                    result_dict = {
                        'd': d,
                        'eps': eps,
                        'sample_size': size_info,
                        'time': t_mc,
                        'memory': peak_memory,
                        'max_err': errors['max'],
                        'max_validation': errors['max_validation'],
                        'mean_err': errors['mean'],
                        'rmse': errors['rmse'],
                        'conv_rate': conv_rate,
                        'valid_percentage': errors['valid_percentage']
                    }

                    # Add error estimate info if available
                    if error_estimates is not None:
                        result_dict['mc_error_est'] = np.mean(error_estimates)

                    # Add to results list
                    results_list.append(result_dict)

                    # Print results
                    if verbose:
                        size_display = f"MC({n_samples})"

                        print(f"{d:2d} | {eps:6.3f} | {size_display:>12} | {t_mc:8.3f} | {peak_memory:6.2f} | "
                              f"{errors['max']:8.2e} | {errors['max_validation']:8.2e} | {errors['mean']:8.2e} | {errors['rmse']:8.2e} | "
                              f"{conv_rate if not np.isnan(conv_rate) else 'N/A':>9}")

                    # Save for convergence rate calculation in the next iteration
                    prev_eps[d] = eps
                    prev_errors[d] = errors['max_validation']

                    # Create visualizations for 1D
                    if d == 1:
                        plot_1d_results(x_ranges, f, s_arrs, U_mc, U_ana, eps, func_name, mc_method, n_samples)
                        plt.savefig(f'{func_name}_1d_mc_{mc_method}_eps{eps}_N{n_samples}.png', dpi=300)
                        plt.close()

                    # Free memory
                    del U_mc
                    if error_estimates is not None:
                        del error_estimates
                    gc.collect()

                except Exception as e:
                    if verbose:
                        print(f"{d:2d} | {eps:6.3f} | {'ERROR':>12} | {'ERROR':>8} | {'ERROR':>6} | "
                              f"{'ERROR':>8} | {'ERROR':>8} | {'ERROR':>8} | {'ERROR':>8} | {'ERROR':>9}")
                        print(f"  Error: {e}")

                    # Add error entry to results
                    results_list.append({
                        'd': d,
                        'eps': eps,
                        'sample_size': f"MC({n_samples})",
                        'time': np.nan,
                        'memory': np.nan,
                        'max_err': np.nan,
                        'max_validation': np.nan,
                        'mean_err': np.nan,
                        'rmse': np.nan,
                        'conv_rate': np.nan,
                        'valid_percentage': np.nan
                    })

            # Free analytical solution to save memory
            del U_ana
            gc.collect()

        except Exception as e:
            if verbose:
                print(f"Error with dimension {d}: {e}")

    # Convert results list to DataFrame
    results_df = pd.DataFrame(results_list)

    # Create a nice table for display
    if verbose:
        print("\nResults Table:")
        # Use only the requested columns for display
        display_df = results_df[columns_to_display].copy()
        print(tabulate(display_df, headers='keys', tablefmt='grid', showindex=False, floatfmt='.3e'))

    # Free memory before returning
    free_memory()

    return results_df

# ---------- Main function -----------------------------------------
def main():
    # Define dimensions and epsilon values to test
    dimensions = [1, 2, 3, 4, 5, 6, 7, 8]  # Can go to higher dimensions with Monte Carlo
    eps_values = [0.001, 0.01, 0.1, 0.5]

    # Dictionary to store all results
    all_results = {}

    # Interactive selection of columns to display
    print("Available columns to display in result tables:")
    all_columns = ['d', 'eps', 'sample_size', 'time', 'memory',
                   'max_err', 'max_validation', 'mean_err', 'rmse', 'conv_rate', 'valid_percentage']

    for i, col in enumerate(all_columns):
        print(f"{i+1}. {col}")

    try:
        selected_indices = input("\nEnter column numbers to display (comma-separated, e.g., 1,2,3,4): ")
        selected_indices = [int(idx.strip()) - 1 for idx in selected_indices.split(',')]
        columns_to_display = [all_columns[idx] for idx in selected_indices if 0 <= idx < len(all_columns)]

        # If no valid selections, use default
        if not columns_to_display:
            print("No valid columns selected, using default columns")
            columns_to_display = ['d', 'eps', 'sample_size', 'time', 'memory',
                                 'max_err', 'max_validation', 'mean_err', 'rmse']
    except Exception as e:
        print(f"Error parsing column selection: {e}")
        print("Using default columns")
        columns_to_display = ['d', 'eps', 'sample_size', 'time', 'memory',
                             'max_err', 'max_validation', 'mean_err', 'rmse']

    print(f"\nSelected columns: {', '.join(columns_to_display)}")

    # Monte Carlo configuration
    print("\nMonte Carlo configuration:")
    print("1. basic - Standard uniform sampling")
    print("2. quasi - Quasi-Monte Carlo with Sobol sequences (default)")
    print("3. importance - Importance sampling based on integrand values")
    print("4. adaptive - Adaptive refinement of sampling in high-contribution regions")

    try:
        mc_method_idx = input("\nSelect Monte Carlo method (1-4, default=2): ").strip()
        mc_methods = ['basic', 'quasi', 'importance', 'adaptive']

        if mc_method_idx == '':
            mc_method = 'quasi'  # Default
        else:
            try:
                mc_method = mc_methods[int(mc_method_idx) - 1]
            except (ValueError, IndexError):
                print("Invalid selection, using quasi-Monte Carlo")
                mc_method = 'quasi'
    except Exception:
        mc_method = 'quasi'
        print("Invalid input, using quasi-Monte Carlo")

    try:
        n_samples_input = input("\nNumber of Monte Carlo samples (default: 100000): ").strip()
        if n_samples_input == '':
            n_samples = 100000  # Default
        else:
            n_samples = int(n_samples_input)
            if n_samples <= 0:
                n_samples = 100000
                print("Invalid number of samples, using default: 100000")
    except Exception:
        n_samples = 100000
        print("Invalid input, using default: 100000 samples")

    print(f"\nUsing {mc_method} Monte Carlo with {n_samples} samples")

    # Run benchmarks for both function types
    for func in ['neg_log', 'neg_entropy']:
        try:
            print(f"\n=== Running benchmark for {func} ===")

            # Run comprehensive benchmark
            results_df = run_comprehensive_benchmark(
                func_name=func,
                dimensions=dimensions,
                eps_values=eps_values,
                columns_to_display=columns_to_display,
                mc_method=mc_method,
                n_samples=n_samples
            )

            # Store results
            all_results[func] = results_df

            # Save results to CSV
            results_df.to_csv(f'{func}_mc_{mc_method}_N{n_samples}_results.csv', index=False)

            # Free memory
            free_memory()

        except Exception as e:
            print(f"Error running benchmark for {func}: {e}")

    # Create combined visualization for convergence rates
    try:
        plt.figure(figsize=(12, 8))

        colors = {
            'neg_log': 'blue',
            'neg_entropy': 'red'
        }

        markers = {
            1: 'o', 2: '^', 3: 's', 4: 'D', 5: 'p',
            6: '*', 7: 'X', 8: 'h', 9: '+', 10: 'x'
        }

        for func, results_df in all_results.items():
            # Group by dimension
            for d, group in results_df.groupby('d'):
                if len(group) < 2:  # Skip if not enough points for a line
                    continue

                # Extract values
                eps_values = group['eps'].values
                # Use max_validation instead of rmse for convergence plot
                err_values = group['max_validation'].values

                # Skip if any error is NaN
                if np.any(np.isnan(err_values)):
                    continue

                # Plot convergence
                label = f"{func.capitalize()}, d={d}, {mc_method} MC"
                plt.loglog(eps_values, err_values,
                           marker=markers.get(d, 'o'),
                           color=colors.get(func, 'black'),
                           label=label, alpha=0.8)

        # Add reference slopes
        x_ref = np.array([min(eps_values), max(eps_values)])
        y_ref1 = 0.1 * x_ref  # O(ε)
        y_ref2 = 0.1 * x_ref**2  # O(ε²)
        y_ref_mc = 0.1 * np.ones_like(x_ref)  # O(1/√N), independent of ε

        plt.loglog(x_ref, y_ref1, 'k--', label='O(ε)', alpha=0.5)
        plt.loglog(x_ref, y_ref2, 'k:', label='O(ε²)', alpha=0.5)
        plt.loglog(x_ref, y_ref_mc, 'k-.', label=f'O(1/√N), N={n_samples}', alpha=0.5)

        plt.xlabel('ε (smoothing parameter)')
        plt.ylabel('Maximum Error (Validation Set)')
        plt.title(f'Convergence of Monte Carlo Approximation ({mc_method.capitalize()})')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True)
        plt.tight_layout()

        plt.savefig(f'convergence_mc_{mc_method}_N{n_samples}.png', dpi=300)
        plt.close()

    except Exception as e:
        print(f"Error creating convergence visualization: {e}")

    # Free memory again
    free_memory()


if __name__ == "__main__":
    main()

Available columns to display in result tables:
1. d
2. eps
3. sample_size
4. time
5. memory
6. max_err
7. max_validation
8. mean_err
9. rmse
10. conv_rate
11. valid_percentage

Selected columns: d

Monte Carlo configuration:
1. basic - Standard uniform sampling
2. quasi - Quasi-Monte Carlo with Sobol sequences (default)
3. importance - Importance sampling based on integrand values
4. adaptive - Adaptive refinement of sampling in high-contribution regions

Using quasi Monte Carlo with 100000 samples

=== Running benchmark for neg_log ===

===== BENCHMARK FOR NEG_LOG (MONTE CARLO: QUASI, 100000 SAMPLES) =====

 d |    eps |  sample size |  time(s) |     MB |  max err |  max val | mean err |     RMSE | conv rate
------------------------------------------------------------------------------------------------------


/usr/local/lib/python3.11/dist-packages/scipy/stats/_qmc.py:993: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)


 1 |  0.001 |   MC(100000) |    1.690 | 139.43 | 1.98e-01 | 3.91e-03 | 1.30e-02 | 4.43e-02 |       N/A
 1 |  0.010 |   MC(100000) |    1.714 |   6.13 | 2.17e-01 | 2.76e-02 | 3.14e-02 | 5.33e-02 | 0.8487293673999331
 1 |  0.100 |   MC(100000) |    1.711 |   6.14 | 2.21e-01 | 1.61e-01 | 1.17e-01 | 1.30e-01 | 0.7643840993965804
 1 |  0.500 |   MC(100000) |    1.793 |   6.13 | 6.38e-01 | 4.06e-01 | 3.10e-01 | 3.55e-01 | 0.5767909402722317
------------------------------------------------------------------------------------------------------
 2 |  0.001 |   MC(100000) |    3.833 |   9.26 | 3.95e-01 | 7.95e-03 | 2.61e-02 | 6.54e-02 |       N/A
 2 |  0.010 |   MC(100000) |    3.936 |   9.26 | 4.33e-01 | 5.48e-02 | 6.28e-02 | 8.75e-02 | 0.8381800891494996
 2 |  0.100 |   MC(100000) |    3.361 |   9.26 | 4.42e-01 | 3.21e-01 | 2.18e-01 | 2.39e-01 | 0.7680961755329327
 2 |  0.500 |   MC(100000) |    3.365 |   9.29 | 1.28e+00 | 8.12e-01 | 4.33e-01 | 5.20e-01 | 0.5764520846989345
-------------------

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
import matplotlib.colors as colors
import psutil
import os
import gc
import pandas as pd
from tabulate import tabulate
import tracemalloc
from scipy.stats import qmc

# ---------- Memory tracking functions ------------------------------
def start_memory_tracking():
    """Start tracking memory allocations"""
    gc.collect()  # Force garbage collection
    tracemalloc.start()
    return tracemalloc.get_traced_memory()[0] / (1024 * 1024)  # Current memory in MB

def get_peak_memory():
    """Get peak memory usage in MB since tracking started"""
    current, peak = tracemalloc.get_traced_memory()
    return peak / (1024 * 1024)  # Peak memory in MB

def stop_memory_tracking():
    """Stop tracking memory allocations"""
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    return peak / (1024 * 1024)  # Peak memory in MB

def free_memory():
    """Aggressively free memory"""
    gc.collect()  # Collect garbage
    if hasattr(plt, 'close'):
        plt.close('all')  # Close all matplotlib figures

    # Try to clear large variables from memory
    for name in list(globals().keys()):
        if name.startswith('__'):
            continue
        obj = globals()[name]
        if isinstance(obj, np.ndarray) and obj.size > 1000000:
            del globals()[name]

    gc.collect()  # Collect garbage again after deleting variables

# ---------- Test functions -----------------------------------------
def neg_log(d):
    """d-dimensional negative logarithm function: u(x) = -sum(log(x_i))
    Domain: x_i > 0
    LF transform: u*(s) = -d - sum(log(-s_i)) for s_i < 0
    """
    def func(*args):
        # Handle domain constraints - inputs should be positive
        result = 0
        for x in args:
            x_safe = np.maximum(x, 1e-10)  # Avoid log(0)
            result -= np.log(x_safe)
        return result
    return func

def neg_entropy(d):
    """d-dimensional negative entropy function: u(x) = sum(x_i * log(x_i))
    Domain: x_i > 0
    LF transform: u*(s) = sum(exp(s_i - 1))
    """
    def func(*args):
        result = 0
        for x in args:
            # Handle domain constraints and avoid singularities
            x_safe = np.maximum(x, 1e-10)
            x_log_x = x_safe * np.log(x_safe)
            # Set values to 0 for x close to 0 (lim x→0 x log(x) = 0)
            x_log_x = np.where(x_safe < 1e-8, 0, x_log_x)
            result += x_log_x
        return result
    return func

# ---------- Analytical LF transforms for validation ----------------
def neg_log_transform_analytical(s_arrs):
    """
    Analytical LF transform for negative log function:
    f*(s) = -d - sum(log(-s_i)) for s_i < 0
    """
    d = len(s_arrs)
    S = np.meshgrid(*s_arrs, indexing='ij', sparse=False)

    # Initialize with -d term
    result = np.full(tuple(len(s) for s in s_arrs), -d, dtype=float)

    # Add log terms for each dimension
    for i in range(d):
        # Apply only where s < 0 (domain constraint)
        valid_mask = S[i] < 0
        # Set large negative values for invalid regions
        log_term = np.log(-S[i])
        result = np.where(valid_mask, result - log_term, -np.inf)

    return result

def neg_entropy_transform_analytical(s_arrs):
    """
    Analytical LF transform for negative entropy function:
    f*(s) = sum(exp(s_i - 1))
    """
    d = len(s_arrs)
    S = np.meshgrid(*s_arrs, indexing='ij', sparse=False)

    # Initialize with zeros
    result = np.zeros(tuple(len(s) for s in s_arrs), dtype=float)

    # Add exp terms for each dimension
    for i in range(d):
        result += np.exp(S[i] - 1)

    return result

# ---------- Monte Carlo LF Transform Methods ----------------------
def lf_transform_monte_carlo(x_ranges, f, s_arrs, eps=0.1, n_samples=100000, method='quasi'):
    """
    Monte Carlo approximation of Legendre-Fenchel transform.

    Parameters:
    x_ranges : list of tuples
        Range for each dimension of x (min, max)
    f : function
        The function to transform
    s_arrs : list of arrays
        Grid points in dual space (one array per dimension)
    eps : float
        Smoothing parameter
    n_samples : int
        Number of Monte Carlo samples
    method : str
        Monte Carlo method to use: 'basic', 'quasi', 'importance', or 'adaptive'

    Returns:
    numpy.ndarray
        The approximate Legendre-Fenchel transform f*_eps(s)
    """
    # Get dimension
    d = len(x_ranges)

    # Create output array for the transform
    result = np.empty([len(s) for s in s_arrs])

    # Create separate error estimates array if needed
    error_estimates = None
    if method in ['basic', 'quasi']:
        error_estimates = np.empty_like(result)

    # Calculate domain volume
    domain_volume = np.prod([x_max - x_min for x_min, x_max in x_ranges])

    # Generate samples based on the selected method
    if method == 'basic':
        # Simple uniform random sampling
        samples = np.array([np.random.uniform(x_min, x_max, n_samples)
                           for x_min, x_max in x_ranges]).T

    elif method == 'quasi':
        # Quasi-Monte Carlo with Sobol sequences for better coverage
        # Initialize Sobol sequence generator
        sampler = qmc.Sobol(d=d, scramble=True)

        # Generate samples in [0, 1]
        unit_samples = sampler.random(n_samples)

        # Scale to the domain range
        samples = np.zeros((n_samples, d))
        for i in range(d):
            x_min, x_max = x_ranges[i]
            samples[:, i] = unit_samples[:, i] * (x_max - x_min) + x_min

    elif method == 'importance' or method == 'adaptive':
        # For importance sampling, we need an initial set of samples to identify high-contribution regions
        # Start with uniform samples
        initial_samples = np.array([np.random.uniform(x_min, x_max, min(10000, n_samples // 10))
                                   for x_min, x_max in x_ranges]).T

        # Pre-compute function values
        f_values_initial = np.array([f(*x) for x in initial_samples])

        # For each output point, we'll generate specialized importance samples
        # This makes this method more computationally intensive but potentially more accurate
        samples_per_point = max(1000, n_samples // (np.prod([len(s) for s in s_arrs])))

    else:
        raise ValueError(f"Unknown Monte Carlo method: {method}")

    # For basic and quasi-Monte Carlo, pre-compute function values for all samples
    if method in ['basic', 'quasi']:
        f_values = np.array([f(*x) for x in samples])

    # Iterate through the output grid
    it = np.nditer(result, flags=['multi_index'], op_flags=['writeonly'])
    while not it.finished:
        # Get current slope point
        s_point = [s_arrs[i][it.multi_index[i]] for i in range(d)]

        if method in ['basic', 'quasi']:
            # Compute inner products for all samples
            inner_products = np.sum([s_point[i] * samples[:, i] for i in range(d)], axis=0)

            # Compute exponents
            exponents = (inner_products - f_values) / eps

            # Shift to prevent overflow
            max_exp = np.max(exponents)
            exponents -= max_exp

            # Compute Monte Carlo approximation
            mc_sum = np.sum(np.exp(exponents))
            mc_avg = mc_sum / n_samples

            # Apply scaling and logarithm
            it[0] = eps * (np.log(domain_volume * mc_avg) + max_exp)

            # Calculate error estimate (for standard Monte Carlo)
            # Standard error = sigma / sqrt(N)
            if error_estimates is not None and method == 'basic':
                variance = np.var(np.exp(exponents)) * (domain_volume**2)
                std_error = np.sqrt(variance / n_samples)
                error_estimates[it.multi_index] = eps * std_error / (domain_volume * mc_avg)


        elif method == 'importance':
            # For importance sampling, we want to sample more heavily in regions where the integrand is large
            # Compute integrand values for initial samples for the current s_point
            inner_products_init = np.sum([s_point[i] * initial_samples[:, i] for i in range(d)], axis=0)
            integrand_values = np.exp((inner_products_init - f_values_initial) / eps)

            # Normalize to create a probability distribution
            if np.sum(integrand_values) > 0:
                prob_dist = integrand_values / np.sum(integrand_values)
            else:
                # If all values are zero (underflow), use uniform distribution
                prob_dist = np.ones_like(integrand_values) / len(integrand_values)

            # Generate importance samples for this specific s_point
            importance_indices = np.random.choice(
                range(len(initial_samples)),
                size=samples_per_point,
                replace=True,
                p=prob_dist
            )
            importance_samples = initial_samples[importance_indices]

            # Compute function values for importance samples
            f_values_importance = f_values_initial[importance_indices]

            # Compute integrand with importance sampling correction
            inner_products = np.sum([s_point[i] * importance_samples[:, i] for i in range(d)], axis=0)
            exponents = (inner_products - f_values_importance) / eps

            # Apply importance sampling correction: divide by probability used for sampling
            importance_weights = 1.0 / (prob_dist[importance_indices] * len(prob_dist))

            # Avoid overflow
            max_exp = np.max(exponents)
            exponents -= max_exp
            weighted_sum = np.sum(np.exp(exponents) * importance_weights)

            # Compute final result
            it[0] = eps * (np.log(domain_volume * weighted_sum / samples_per_point) + max_exp)

        elif method == 'adaptive':
            # Similar to importance sampling but with adaptive refinement
            # First pass with initial samples
            inner_products_init = np.sum([s_point[i] * initial_samples[:, i] for i in range(d)], axis=0)
            integrand_values = np.exp((inner_products_init - f_values_initial) / eps)

            # Identify regions needing refinement (high integrand value or high variance)
            if len(integrand_values) > 0:
                threshold = np.percentile(integrand_values, 95)  # Focus on top 5%
                high_contribution_indices = np.where(integrand_values > threshold)[0]

                if len(high_contribution_indices) > 0:
                    # Extract high-contribution samples
                    high_samples = initial_samples[high_contribution_indices]

                    # Define a refined sampling region around these samples
                    refined_ranges = []
                    for dim in range(d):
                        min_val = max(x_ranges[dim][0], np.min(high_samples[:, dim]) - 0.1 * (x_ranges[dim][1] - x_ranges[dim][0]))
                        max_val = min(x_ranges[dim][1], np.max(high_samples[:, dim]) + 0.1 * (x_ranges[dim][1] - x_ranges[dim][0]))
                        refined_ranges.append((min_val, max_val))

                    # Generate refined samples
                    refined_samples = np.array([np.random.uniform(r_min, r_max, samples_per_point)
                                              for r_min, r_max in refined_ranges]).T

                    # Compute function values for refined samples
                    f_values_refined = np.array([f(*x) for x in refined_samples])

                    # Compute integrand for refined samples
                    inner_products_refined = np.sum([s_point[i] * refined_samples[:, i] for i in range(d)], axis=0)
                    exponents_refined = (inner_products_refined - f_values_refined) / eps

                    # Calculate refined volume
                    refined_volume = np.prod([r_max - r_min for r_min, r_max in refined_ranges])
                    ratio = refined_volume / domain_volume

                    # Combine with initial estimate (weighted by volume ratio)
                    max_exp_init = np.max(inner_products_init - f_values_initial) / eps if len(inner_products_init) > 0 else 0
                    init_sum = np.sum(np.exp(((inner_products_init - f_values_initial) / eps) - max_exp_init))

                    max_exp_refined = np.max(exponents_refined) if len(exponents_refined) > 0 else 0
                    refined_sum = np.sum(np.exp(exponents_refined - max_exp_refined))

                    # Combine estimates with appropriate volume scaling
                    max_exp_combined = max(max_exp_init, max_exp_refined)
                    init_contribution = init_sum * np.exp(max_exp_init - max_exp_combined) * (1 - ratio)
                    refined_contribution = refined_sum * np.exp(max_exp_refined - max_exp_combined) * ratio

                    combined_sum = init_contribution + refined_contribution
                    it[0] = eps * (np.log(domain_volume * combined_sum / samples_per_point) + max_exp_combined)
                else:
                    # Fall back to basic MC if no high-contribution regions found
                    max_exp = np.max((inner_products_init - f_values_initial) / eps)
                    mc_sum = np.sum(np.exp(((inner_products_init - f_values_initial) / eps) - max_exp))
                    mc_avg = mc_sum / len(initial_samples)
                    it[0] = eps * (np.log(domain_volume * mc_avg) + max_exp)
            else:
                # No valid samples, fall back to analytical approximation if possible
                it[0] = 0  # Default value

        it.iternext()

    # Return both result and error estimates if available
    if error_estimates is not None:
        return result, error_estimates
    return result

# ---------- Error calculation functions ---------------------
def calculate_errors(U_mc, U_ana, validation_set=None):
    """
    Calculate various error metrics between Monte Carlo approximation and analytical solution.

    Parameters:
    U_mc : numpy.ndarray
        Monte Carlo approximation
    U_ana : numpy.ndarray
        Analytical solution
    validation_set : tuple, optional
        Tuple of slice objects defining a validation set region

    Returns:
    dict
        Dictionary of error metrics
    """
    # Get valid points (exclude -inf values)
    valid_mask = ~np.isinf(U_ana) & ~np.isinf(U_mc) & ~np.isnan(U_ana) & ~np.isnan(U_mc)

    if np.sum(valid_mask) == 0:
        return {
            'max': np.nan,
            'mean': np.nan,
            'median': np.nan,
            'max_validation': np.nan,
            'rmse': np.nan,
            'valid_percentage': 0
        }

    # Extract valid points
    U_mc_valid = U_mc[valid_mask]
    U_ana_valid = U_ana[valid_mask]

    # Calculate absolute errors
    abs_errors = np.abs(U_mc_valid - U_ana_valid)

    # Calculate max error over validation set if provided
    max_validation_error = np.nan
    if validation_set is not None:
        # Apply validation set slices
        validation_mask = valid_mask.copy()
        if isinstance(validation_set, tuple):
            validation_mask[validation_set] &= valid_mask[validation_set]

            # Check if there are valid points in the validation set
            if np.any(validation_mask):
                # Calculate max error over validation set
                validation_errors = np.abs(U_mc[validation_mask] - U_ana[validation_mask])
                max_validation_error = np.max(validation_errors)
    else:
        # If no validation set provided, use a central region
        # (exclude 20% of points from each edge)
        shape = valid_mask.shape
        ndim = len(shape)

        validation_mask = valid_mask.copy()
        for d in range(ndim):
            size = shape[d]
            margin = max(1, int(size * 0.2))

            # Create a slice for this dimension
            idx = tuple(slice(margin, size - margin) if i == d else slice(None)
                        for i in range(ndim))

            # Update validation mask
            subregion = np.zeros_like(valid_mask, dtype=bool)
            subregion[idx] = True
            validation_mask &= subregion

        # Calculate max error over validation set
        if np.any(validation_mask):
            validation_errors = np.abs(U_mc[validation_mask] - U_ana[validation_mask])
            max_validation_error = np.max(validation_errors)

    # Calculate RMSE
    squared_errors = (U_mc_valid - U_ana_valid)**2
    rmse = np.sqrt(np.mean(squared_errors))  # Root Mean Square Error

    # Calculate percentage of valid points
    valid_percentage = 100.0 * np.sum(valid_mask) / valid_mask.size

    # Free memory for large arrays
    del validation_mask, valid_mask
    gc.collect()

    return {
        'max': np.max(abs_errors),
        'mean': np.mean(abs_errors),
        'median': np.median(abs_errors),
        'max_validation': max_validation_error,
        'rmse': rmse,
        'valid_percentage': valid_percentage
    }

# ---------- Plot functions -----------------------------------------
def plot_1d_results(x_ranges, f, s_arrays, u_star_mc, u_star_analytical, eps, func_name, mc_method, n_samples):
    """Plot results for 1D transform."""
    plt.figure(figsize=(15, 5))

    # Sample points to plot original function
    x = np.linspace(x_ranges[0][0], x_ranges[0][1], 100)
    u_x = np.array([f(xi) for xi in x])

    # Plot original function
    plt.subplot(1, 3, 1)
    plt.plot(x, u_x)
    plt.xlabel('x')
    plt.ylabel('u(x)')
    plt.title(f'Original {func_name.capitalize()} Function')
    plt.grid(True)

    # Plot transforms
    plt.subplot(1, 3, 2)
    s = s_arrays[0]
    plt.plot(s, u_star_mc, 'g-.', label=f'{mc_method.capitalize()} MC (ε={eps}, N={n_samples})')
    plt.plot(s, u_star_analytical, 'k-', label='Analytical')

    plt.xlabel('s')
    plt.ylabel('u*(s)')
    plt.title('Legendre Transform Comparison')
    plt.legend()
    plt.grid(True)

    # Plot error
    plt.subplot(1, 3, 3)
    error = np.abs(u_star_mc - u_star_analytical)
    plt.semilogy(s, error, 'r-', label='|MC-Analytical|')

    # Add reference line showing MC error scaling O(1/√N)
    ref_line = 1.0 / np.sqrt(n_samples) * np.ones_like(s)
    plt.semilogy(s, ref_line, 'k--', label=f'1/√N reference (N={n_samples})')

    plt.xlabel('s')
    plt.ylabel('Absolute Error')
    plt.title('Error vs Analytical')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()

# ---------- Run comprehensive benchmark ---------------------------
def run_comprehensive_benchmark(func_name, dimensions=[1, 2, 3, 4, 5], eps_values=[0.001, 0.01, 0.1, 0.5],
                               verbose=True, columns_to_display=None, mc_method='quasi', n_samples=100000):
    """
    Run comprehensive benchmark for the specified function with multiple dimensions and epsilon values.

    Parameters:
    func_name : str
        Name of the function to test ('neg_log' or 'neg_entropy')
    dimensions : list of int
        List of dimensions to test
    eps_values : list of float
        List of epsilon values to test
    verbose : bool
        Whether to print progress information
    columns_to_display : list of str
        List of column names to display in the final table
    mc_method : str
        Monte Carlo method to use ('basic', 'quasi', 'importance', 'adaptive')
    n_samples : int
        Number of Monte Carlo samples to use

    Returns:
    pandas.DataFrame
        DataFrame containing benchmark results
    """
    # Select function based on name
    if func_name == 'neg_log':
        f_creator = neg_log
        analytical_transform = neg_log_transform_analytical
        # Domain constraints: x > 0, s < 0
        x_range = (0.1, 5.0)
        s_range = (-5.0, -0.1)
    elif func_name == 'neg_entropy':
        f_creator = neg_entropy
        analytical_transform = neg_entropy_transform_analytical
        # Domain constraints: x > 0, s can be any real number
        x_range = (0.1, 5.0)
        s_range = (-3.0, 3.0)
    else:
        raise ValueError(f"Unknown function type: {func_name}")

    # Create results list to convert to DataFrame later
    results_list = []

    # If no columns specified, use default set
    if columns_to_display is None:
        columns_to_display = ['d', 'eps', 'sample_size', 'time', 'memory',
                             'max_err', 'max_validation', 'mean_err', 'rmse', 'conv_rate']

    # Print header
    if verbose:
        print(f"\n===== BENCHMARK FOR {func_name.upper()} (MONTE CARLO: {mc_method.upper()}, {n_samples} SAMPLES) =====\n")
        header = f"{'d':>2} | {'eps':>6} | {'sample size':>12} | {'time(s)':>8} | {'MB':>6} | {'max err':>8} | {'max val':>8} | {'mean err':>8} | {'RMSE':>8} | {'conv rate':>9}"
        print(header)
        print("-" * len(header))

    # Previous errors for convergence rate calculation
    prev_eps = {}
    prev_errors = {}

    # Loop through dimensions
    for d in dimensions:
        # Create evaluation grids for output and analytical comparison
        n_eval_pts = min(20, 100 // d)  # Reduce grid size for high dimensions
        s_arrs = [np.linspace(s_range[0], s_range[1], n_eval_pts) for _ in range(d)]
        x_ranges = [(x_range[0], x_range[1]) for _ in range(d)]
        f = f_creator(d)

        # Calculate analytical solution for comparison
        try:
            # Calculate analytical solution
            U_ana = analytical_transform(s_arrs)

            # Add a horizontal rule before each dimension
            if d > dimensions[0] and verbose:
                print("-" * len(header))

            # Loop through epsilon values
            for eps in sorted(eps_values):
                try:
                    # Measure memory and time using tracemalloc
                    start_memory_tracking()

                    t0 = time.perf_counter()

                    # Use Monte Carlo method - handle both result and possible error estimates
                    result_data = lf_transform_monte_carlo(
                        x_ranges=x_ranges,
                        f=f,
                        s_arrs=s_arrs,
                        eps=eps,
                        n_samples=n_samples,
                        method=mc_method
                    )

                    # Handle either single result or (result, error_estimates) tuple
                    if isinstance(result_data, tuple) and len(result_data) == 2:
                        U_mc, error_estimates = result_data
                    else:
                        U_mc = result_data
                        error_estimates = None

                    t_mc = time.perf_counter() - t0

                    # Get peak memory usage
                    peak_memory = get_peak_memory()
                    stop_memory_tracking()

                    # Define validation set (central region)
                    validation_set = None  # Using default central region

                    # Calculate errors
                    errors = calculate_errors(U_mc, U_ana, validation_set)

                    # Calculate convergence rate if possible
                    conv_rate = np.nan
                    if d in prev_eps and d in prev_errors and prev_eps[d] is not None:
                        if prev_errors[d] > 0 and errors['max_validation'] > 0:  # Avoid division by zero or log(0)
                            conv_rate = np.log(errors['max_validation']/prev_errors[d]) / np.log(eps/prev_eps[d])

                    # Store results in dictionary
                    size_info = f"MC({n_samples})"

                    result_dict = {
                        'd': d,
                        'eps': eps,
                        'sample_size': size_info,
                        'time': t_mc,
                        'memory': peak_memory,
                        'max_err': errors['max'],
                        'max_validation': errors['max_validation'],
                        'mean_err': errors['mean'],
                        'rmse': errors['rmse'],
                        'conv_rate': conv_rate,
                        'valid_percentage': errors['valid_percentage']
                    }

                    # Add error estimate info if available
                    if error_estimates is not None:
                        result_dict['mc_error_est'] = np.mean(error_estimates)

                    # Add to results list
                    results_list.append(result_dict)

                    # Print results
                    if verbose:
                        size_display = f"MC({n_samples})"

                        print(f"{d:2d} | {eps:6.3f} | {size_display:>12} | {t_mc:8.3f} | {peak_memory:6.2f} | "
                              f"{errors['max']:8.2e} | {errors['max_validation']:8.2e} | {errors['mean']:8.2e} | {errors['rmse']:8.2e} | "
                              f"{conv_rate if not np.isnan(conv_rate) else 'N/A':>9}")

                    # Save for convergence rate calculation in the next iteration
                    prev_eps[d] = eps
                    prev_errors[d] = errors['max_validation']

                    # Create visualizations for 1D
                    if d == 1:
                        plot_1d_results(x_ranges, f, s_arrs, U_mc, U_ana, eps, func_name, mc_method, n_samples)
                        plt.savefig(f'{func_name}_1d_mc_{mc_method}_eps{eps}_N{n_samples}.png', dpi=300)
                        plt.close()

                    # Free memory
                    del U_mc
                    if error_estimates is not None:
                        del error_estimates
                    gc.collect()

                except Exception as e:
                    if verbose:
                        print(f"{d:2d} | {eps:6.3f} | {'ERROR':>12} | {'ERROR':>8} | {'ERROR':>6} | "
                              f"{'ERROR':>8} | {'ERROR':>8} | {'ERROR':>8} | {'ERROR':>8} | {'ERROR':>9}")
                        print(f"  Error: {e}")

                    # Add error entry to results
                    results_list.append({
                        'd': d,
                        'eps': eps,
                        'sample_size': f"MC({n_samples})",
                        'time': np.nan,
                        'memory': np.nan,
                        'max_err': np.nan,
                        'max_validation': np.nan,
                        'mean_err': np.nan,
                        'rmse': np.nan,
                        'conv_rate': np.nan,
                        'valid_percentage': np.nan
                    })

            # Free analytical solution to save memory
            del U_ana
            gc.collect()

        except Exception as e:
            if verbose:
                print(f"Error with dimension {d}: {e}")

    # Convert results list to DataFrame
    results_df = pd.DataFrame(results_list)

    # Create a nice table for display
    if verbose:
        print("\nResults Table:")
        # Use only the requested columns for display
        display_df = results_df[columns_to_display].copy()
        print(tabulate(display_df, headers='keys', tablefmt='grid', showindex=False, floatfmt='.3e'))

    # Free memory before returning
    free_memory()

    return results_df

# ---------- Main function -----------------------------------------
def main():
    # Define dimensions and epsilon values to test
    dimensions = [1, 2, 3, 4, 5, 6, 7, 8]  # Can go to higher dimensions with Monte Carlo
    eps_values = [0.001, 0.01, 0.1, 0.5]

    # Dictionary to store all results
    all_results = {}

    # Interactive selection of columns to display
    print("Available columns to display in result tables:")
    all_columns = ['d', 'eps', 'sample_size', 'time', 'memory',
                   'max_err', 'max_validation', 'mean_err', 'rmse', 'conv_rate', 'valid_percentage']

    for i, col in enumerate(all_columns):
        print(f"{i+1}. {col}")

    try:
        selected_indices = input("\nEnter column numbers to display (comma-separated, e.g., 1,2,3,4): ")
        selected_indices = [int(idx.strip()) - 1 for idx in selected_indices.split(',')]
        columns_to_display = [all_columns[idx] for idx in selected_indices if 0 <= idx < len(all_columns)]

        # If no valid selections, use default
        if not columns_to_display:
            print("No valid columns selected, using default columns")
            columns_to_display = ['d', 'eps', 'sample_size', 'time', 'memory',
                                 'max_err', 'max_validation', 'mean_err', 'rmse']
    except Exception as e:
        print(f"Error parsing column selection: {e}")
        print("Using default columns")
        columns_to_display = ['d', 'eps', 'sample_size', 'time', 'memory',
                             'max_err', 'max_validation', 'mean_err', 'rmse']

    print(f"\nSelected columns: {', '.join(columns_to_display)}")

    # Monte Carlo configuration
    print("\nMonte Carlo configuration:")
    print("1. basic - Standard uniform sampling")
    print("2. quasi - Quasi-Monte Carlo with Sobol sequences (default)")
    print("3. importance - Importance sampling based on integrand values")
    print("4. adaptive - Adaptive refinement of sampling in high-contribution regions")

    try:
        mc_method_idx = input("\nSelect Monte Carlo method (1-4, default=2): ").strip()
        mc_methods = ['basic', 'quasi', 'importance', 'adaptive']

        if mc_method_idx == '':
            mc_method = 'quasi'  # Default
        else:
            try:
                mc_method = mc_methods[int(mc_method_idx) - 1]
            except (ValueError, IndexError):
                print("Invalid selection, using quasi-Monte Carlo")
                mc_method = 'quasi'
    except Exception:
        mc_method = 'quasi'
        print("Invalid input, using quasi-Monte Carlo")

    try:
        n_samples_input = input("\nNumber of Monte Carlo samples (default: 100000): ").strip()
        if n_samples_input == '':
            n_samples = 100000  # Default
        else:
            n_samples = int(n_samples_input)
            if n_samples <= 0:
                n_samples = 100000
                print("Invalid number of samples, using default: 100000")
    except Exception:
        n_samples = 100000
        print("Invalid input, using default: 100000 samples")

    print(f"\nUsing {mc_method} Monte Carlo with {n_samples} samples")

    # Run benchmarks for both function types
    for func in [ 'neg_entropy']:
        try:
            print(f"\n=== Running benchmark for {func} ===")

            # Run comprehensive benchmark
            results_df = run_comprehensive_benchmark(
                func_name=func,
                dimensions=dimensions,
                eps_values=eps_values,
                columns_to_display=columns_to_display,
                mc_method=mc_method,
                n_samples=n_samples
            )

            # Store results
            all_results[func] = results_df

            # Save results to CSV
            results_df.to_csv(f'{func}_mc_{mc_method}_N{n_samples}_results.csv', index=False)

            # Free memory
            free_memory()

        except Exception as e:
            print(f"Error running benchmark for {func}: {e}")

    # Create combined visualization for convergence rates
    try:
        plt.figure(figsize=(12, 8))

        colors = {
            'neg_log': 'blue',
            'neg_entropy': 'red'
        }

        markers = {
            1: 'o', 2: '^', 3: 's', 4: 'D', 5: 'p',
            6: '*', 7: 'X', 8: 'h', 9: '+', 10: 'x'
        }

        for func, results_df in all_results.items():
            # Group by dimension
            for d, group in results_df.groupby('d'):
                if len(group) < 2:  # Skip if not enough points for a line
                    continue

                # Extract values
                eps_values = group['eps'].values
                # Use max_validation instead of rmse for convergence plot
                err_values = group['max_validation'].values

                # Skip if any error is NaN
                if np.any(np.isnan(err_values)):
                    continue

                # Plot convergence
                label = f"{func.capitalize()}, d={d}, {mc_method} MC"
                plt.loglog(eps_values, err_values,
                           marker=markers.get(d, 'o'),
                           color=colors.get(func, 'black'),
                           label=label, alpha=0.8)

        # Add reference slopes
        x_ref = np.array([min(eps_values), max(eps_values)])
        y_ref1 = 0.1 * x_ref  # O(ε)
        y_ref2 = 0.1 * x_ref**2  # O(ε²)
        y_ref_mc = 0.1 * np.ones_like(x_ref)  # O(1/√N), independent of ε

        plt.loglog(x_ref, y_ref1, 'k--', label='O(ε)', alpha=0.5)
        plt.loglog(x_ref, y_ref2, 'k:', label='O(ε²)', alpha=0.5)
        plt.loglog(x_ref, y_ref_mc, 'k-.', label=f'O(1/√N), N={n_samples}', alpha=0.5)

        plt.xlabel('ε (smoothing parameter)')
        plt.ylabel('Maximum Error (Validation Set)')
        plt.title(f'Convergence of Monte Carlo Approximation ({mc_method.capitalize()})')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True)
        plt.tight_layout()

        plt.savefig(f'convergence_mc_{mc_method}_N{n_samples}.png', dpi=300)
        plt.close()

    except Exception as e:
        print(f"Error creating convergence visualization: {e}")

    # Free memory again
    free_memory()


if __name__ == "__main__":
    main()

Available columns to display in result tables:
1. d
2. eps
3. sample_size
4. time
5. memory
6. max_err
7. max_validation
8. mean_err
9. rmse
10. conv_rate
11. valid_percentage

Selected columns: d, eps, sample_size, time, memory, max_err, max_validation, mean_err, rmse, valid_percentage

Monte Carlo configuration:
1. basic - Standard uniform sampling
2. quasi - Quasi-Monte Carlo with Sobol sequences (default)
3. importance - Importance sampling based on integrand values
4. adaptive - Adaptive refinement of sampling in high-contribution regions

Using quasi Monte Carlo with 100000 samples

=== Running benchmark for neg_entropy ===

===== BENCHMARK FOR NEG_ENTROPY (MONTE CARLO: QUASI, 100000 SAMPLES) =====

 d |    eps |  sample size |  time(s) |     MB |  max err |  max val | mean err |     RMSE | conv rate
------------------------------------------------------------------------------------------------------
 1 |  0.001 |   MC(100000) |    4.379 |  11.61 | 4.42e-01 | 1.43e-02 | 3.78e-02